In [4]:
import json
import re
import time
import requests
import numpy as np
import pandas as pd
import os
import glob
from tqdm import tqdm
from datetime import datetime
from bs4 import BeautifulSoup
from fake_useragent import UserAgent
import warnings
warnings.filterwarnings('ignore')
from random import choice

In [5]:
current_datetime = datetime.now().strftime("%d-%m-%Y")
month_now = datetime.now().month
day_now = datetime.now().day

# Cookies and Headers для парсинга скидки и цены

In [6]:
cookies_discount = {
    'tmr_lvid': '286726132090325c59b36aa6b5e4aeaa',
    'tmr_lvidTS': '1692163620235',
    '_ym_uid': '1692163620496112993',
    '_ym_d': '1692163620',
    'city_id': '118',
    'city_name': '%D0%A1%D0%B0%D0%BD%D0%BA%D1%82-%D0%9F%D0%B5%D1%82%D0%B5%D1%80%D0%B1%D1%83%D1%80%D0%B3',
    'city_full_name': '%D0%A1%D0%B0%D0%BD%D0%BA%D1%82-%D0%9F%D0%B5%D1%82%D0%B5%D1%80%D0%B1%D1%83%D1%80%D0%B3',
    'region_id': '090a0c31-3b91-49a3-aa35-0c784898170b',
    'region_name': '%D0%A1%D0%B0%D0%BD%D0%BA%D1%82-%D0%9F%D0%B5%D1%82%D0%B5%D1%80%D0%B1%D1%83%D1%80%D0%B3',
    'region_subdomain': 'spb',
    '_tt_enable_cookie': '1',
    '_ttp': 'tbKEEA3IkGrgdKpaT6-HRNf0K8h',
    'ccart': 'off',
    'uxs_uid': '898ab7e0-3bf5-11ee-8efe-f5b770db12a1',
    'loggedin': '%D0%AE%D1%80%D0%B8%D0%B9',
    'cseid': 'f4935df6bdac0029832e99cef94a5bc2',
    'sessionid': 'nr24mlml3ze25rk734gkkmgx3yv9ou7x',
    'authorization': 'true',
    'mindboxDeviceUUID': 'c254fa3f-cfcd-4146-9778-65be21c6a4df',
    'directCrm-session': '%7B%22deviceGuid%22%3A%22c254fa3f-cfcd-4146-9778-65be21c6a4df%22%7D',
    'auid': '651660ce-a06e-4b17-bd6f-69feaf135b08:1r5lu8:WYtTZugo2xVE6SqAc5ZSp_W-n1s5ztPHYxPs8YIwGQA',
    '_ga_G74SVHM1ZG': 'GS1.2.1700824600.19.0.1700824600.60.0.0',
    '_ga_GRDJMMMRHQ': 'GS1.2.1700824600.33.0.1700824601.59.0.0',
    '_ga_GRSMPDQ6SE': 'GS1.2.1700824598.88.1.1700825516.60.0.0',
    '_ga_M9ZHPEJXBM': 'GS1.2.1700824599.88.1.1700825517.60.0.0',
    'qr-go-to-the-app-banner': '1',
    'sncid': 'b36ee46b-981d-11e5-add2-0050568f2465:1rBa43:O1Wy8KuwGDddSCM7NMHFsjGhdPaqxcjhdblvWWbVams',
    'clt': 'new_old',
    '_ym_isad': '2',
    '_gid': 'GA1.2.2131018395.1702274408',
    'city_auto_popup_shown': '1',
    '_ym_visorc': 'b',
    '_ga': 'GA1.2.333284150.1692163620',
    '_ga_HJNSJ6NG5J': 'GS1.1.1702276831.120.1.1702280271.20.0.0',
    'digi_uc': 'W1sidiIsIjM0OTMxNCIsMTcwMjI4MTIxMjA0Nl0sWyJ2IiwiODQwMjIiLDE3MDIyODAyMzEzNjZdLFsidiIsIjQ2Nzk3IiwxNzAyMjgwMTI4Mzc3XSxbInYiLCI5OTM3NTQiLDE3MDIyNzgwMTczMjddLFsidiIsIjE0NjUyMSIsMTcwMjI3NzI5NDExM10sWyJ2IiwiOTI2ODAiLDE3MDIyNzY5NDQ1NDRdLFsidiIsIjg4MjEzIiwxNzAxODU5MjE3OTg0XSxbInYiLCI5MTAyMSIsMTcwMTg1OTA2ODA2MF0sWyJ2IiwiOTg3MDUwIiwxNzAxNzc4NDc0NzY5XSxbInYiLCIyNTYyMTciLDE3MDEzNDIzNTc4NTRdXQ==',
}

headers_discount = {
    'AB-TESTS': '{}',
    'Accept': 'application/json, text/javascript, */*; q=0.01',
    'Accept-Language': 'ru-RU,ru;q=0.9,en-US;q=0.8,en;q=0.7',
    'Connection': 'keep-alive',
    'Content-Type': 'application/json',
    # 'Cookie': 'tmr_lvid=286726132090325c59b36aa6b5e4aeaa; tmr_lvidTS=1692163620235; _ym_uid=1692163620496112993; _ym_d=1692163620; city_id=118; city_name=%D0%A1%D0%B0%D0%BD%D0%BA%D1%82-%D0%9F%D0%B5%D1%82%D0%B5%D1%80%D0%B1%D1%83%D1%80%D0%B3; city_full_name=%D0%A1%D0%B0%D0%BD%D0%BA%D1%82-%D0%9F%D0%B5%D1%82%D0%B5%D1%80%D0%B1%D1%83%D1%80%D0%B3; region_id=090a0c31-3b91-49a3-aa35-0c784898170b; region_name=%D0%A1%D0%B0%D0%BD%D0%BA%D1%82-%D0%9F%D0%B5%D1%82%D0%B5%D1%80%D0%B1%D1%83%D1%80%D0%B3; region_subdomain=spb; _tt_enable_cookie=1; _ttp=tbKEEA3IkGrgdKpaT6-HRNf0K8h; ccart=off; uxs_uid=898ab7e0-3bf5-11ee-8efe-f5b770db12a1; loggedin=%D0%AE%D1%80%D0%B8%D0%B9; cseid=f4935df6bdac0029832e99cef94a5bc2; sessionid=nr24mlml3ze25rk734gkkmgx3yv9ou7x; authorization=true; mindboxDeviceUUID=c254fa3f-cfcd-4146-9778-65be21c6a4df; directCrm-session=%7B%22deviceGuid%22%3A%22c254fa3f-cfcd-4146-9778-65be21c6a4df%22%7D; auid=651660ce-a06e-4b17-bd6f-69feaf135b08:1r5lu8:WYtTZugo2xVE6SqAc5ZSp_W-n1s5ztPHYxPs8YIwGQA; _ga_G74SVHM1ZG=GS1.2.1700824600.19.0.1700824600.60.0.0; _ga_GRDJMMMRHQ=GS1.2.1700824600.33.0.1700824601.59.0.0; _ga_GRSMPDQ6SE=GS1.2.1700824598.88.1.1700825516.60.0.0; _ga_M9ZHPEJXBM=GS1.2.1700824599.88.1.1700825517.60.0.0; qr-go-to-the-app-banner=1; sncid=b36ee46b-981d-11e5-add2-0050568f2465:1rBa43:O1Wy8KuwGDddSCM7NMHFsjGhdPaqxcjhdblvWWbVams; clt=new_old; _ym_isad=2; _gid=GA1.2.2131018395.1702274408; city_auto_popup_shown=1; _ym_visorc=b; _ga=GA1.2.333284150.1692163620; _ga_HJNSJ6NG5J=GS1.1.1702276831.120.1.1702280271.20.0.0; digi_uc=W1sidiIsIjM0OTMxNCIsMTcwMjI4MTIxMjA0Nl0sWyJ2IiwiODQwMjIiLDE3MDIyODAyMzEzNjZdLFsidiIsIjQ2Nzk3IiwxNzAyMjgwMTI4Mzc3XSxbInYiLCI5OTM3NTQiLDE3MDIyNzgwMTczMjddLFsidiIsIjE0NjUyMSIsMTcwMjI3NzI5NDExM10sWyJ2IiwiOTI2ODAiLDE3MDIyNzY5NDQ1NDRdLFsidiIsIjg4MjEzIiwxNzAxODU5MjE3OTg0XSxbInYiLCI5MTAyMSIsMTcwMTg1OTA2ODA2MF0sWyJ2IiwiOTg3MDUwIiwxNzAxNzc4NDc0NzY5XSxbInYiLCIyNTYyMTciLDE3MDEzNDIzNTc4NTRdXQ==',
    'Origin': 'https://spb.sunlight.net',
    'Sec-Fetch-Dest': 'empty',
    'Sec-Fetch-Mode': 'cors',
    'Sec-Fetch-Site': 'same-site',
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36',
    'X-Requested-With': 'SunlightFrontendApp',
    'sec-ch-ua': '"Not_A Brand";v="8", "Chromium";v="120", "Google Chrome";v="120"',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-platform': '"Windows"',
}

# params = {
#     'article': '35517',
#     'city_id': '118',
# }

# response = requests.get(
#     'https://api.sunlight.net/v6/catalog/products/card/purchase/',
#     params=params,
#     cookies=cookies_discount,
#     headers=headers_discount,
# )

# Сбор скидок и цен

In [7]:
def get_personal_price(table):
    data_price = []
    for art in tqdm(range(len(table['article']))):
        # prx = choice(proxy_list)
        params = {
            'article': table['article'][art],
            'city_id': '118'
        }
        try:
            price_json = requests.get(
                'https://api.sunlight.net/v6/catalog/products/card/prices/',
                params=params,
                cookies=cookies_discount,
                headers=headers_discount,
                timeout=15,
                verify=False
        )
        except:
            time.sleep(6)
            price_json = requests.get(
                'https://api.sunlight.net/v6/catalog/products/card/prices/',
                params=params,
                cookies=cookies_discount,
                headers=headers_discount,
                timeout=15,
                verify=False
                            )
        try:
            discount_json = requests.get(
                'https://api.sunlight.net/v6/catalog/products/card/purchase/',
                params=params,
                cookies=cookies_discount,
                headers=headers_discount,
                timeout=15,
                verify=False
                            )
        except:
            time.sleep(6)
            discount_json = requests.get(
                'https://api.sunlight.net/v6/catalog/products/card/purchase/',
                params=params,
                cookies=cookies_discount,
                headers=headers_discount,
                timeout=15,
                verify=False
                            )
        discount = []
        try:
            price_old = price_json.json()['content']['default_price']['base_sum']
        except:
            price_old = np.nan
        try:
            price = price_json.json()['content']['default_price'][('result_sum')]
        except:
            price = np.nan
        try:
            price_personal = price_json.json()['content']['default_price']['personal_block']['value']
        except:
            price_personal = np.nan

        try:
            for i in range(len(discount_json.json()['content']['nameplates'])):
                discount.append(discount_json.json()['content']['nameplates'][i]['text']['label'])
        except:
            discount.append(np.nan)
            
        data_price.append({
            'article': table['article'][art],
            'price_old': price_old,
            'price': price,
            'price_personal': price_personal,
            'discount': discount

        })
    return data_price

# Сбор цены, названия, группы, артикула

In [8]:
def get_sunlight(soup):
    try:
        price = soup.find(class_="supreme-product-card__price-discount-price").text
        price = re.sub(r'\D', '', str(price))
    except:
        price = np.nan
    try:
        name_product = soup.find('h1', class_="supreme-product-card__about-title").text
    except:
        name_product = np.nan
    try:
        group_product = soup.find_all('span', {'property': 'name'})[1].text
    except:
        group_product = np.nan
    try:
        article_product = soup.find_all('div', class_="supreme-product-card__product-article")[0].find('span').text
        article_product = re.sub('\D', '', str(article_product))
    except:
        article_product = np.nan
    return price, name_product, group_product, article_product

# Сбор всех параметров словарем

In [9]:
def get_params(soup):
    names = []
    values = []
    leight = len(soup.find_all('span', {'class': 'supreme-product-card-description__item-text'}))
    list_names = soup.find_all('div', {'class': 'supreme-product-card-description__item-title'})[-leight:]
    for i in range(leight):
        try:
            nam = list_names[i].text
            names.append(nam.replace('\n', '').strip())
        except:
            names.append(np.nan)
        try:
            values.append(soup.find_all('span', {'class': 'supreme-product-card-description__item-text'})[i].text)
        except:
            values.append(np.nan)
    sunlight_params = dict(zip(names, values))
    return sunlight_params

# Сбор ссылок

In [10]:
def get_links(url_sunlight):
    home = 'https://sunlight.net'
    links = []
    pages_max = 250
    pages = int(pages_max + 1)
    for i in tqdm(range(1, pages)):
        url = f'{url_sunlight}&page={i}'
        resp = requests.get(url, headers={'User-Agent': UserAgent().chrome}, verify=False, timeout=70)
        soup = BeautifulSoup(resp.text, 'html.parser')
        for link in soup.find_all('div', class_="cl-item js-cl-item"):
            links.append(home + link.find('a', href=True)['href'])
    links = list(map(lambda x: x.replace('#product-reviews', ''), links))
    return links

# Общий сбор данных

In [13]:
def sunlight_parser(links):
    data = []
    click = 0
    for link_pars in tqdm(links):
        # prx = choice(proxy_list)
        try:
            resp = requests.get(link_pars, headers={'User-Agent': UserAgent().chrome}, timeout=70, verify=False)
        except:
            time.sleep(6)
            resp = requests.get(link_pars, headers={'User-Agent': UserAgent().chrome}, timeout=70, verify=False)
        good = BeautifulSoup(resp.text, 'lxml')
        data.append({"date": current_datetime,
                    "competitor": "Sunlight",
                    "name": get_sunlight(good)[1],
                    "group": get_sunlight(good)[2],
                    "params": get_params(good),
                    "price": get_sunlight(good)[0],
                    "article": get_sunlight(good)[3],
                    'url': link_pars
                    })
        if click == 50:
            click = 0
            time.sleep(0.5)
        else:
            continue
    data = pd.DataFrame(data)
    return data

In [9]:
links_sunlight = get_links('https://spb.sunlight.net/catalog/?proba=2494')

  0%|          | 0/250 [00:00<?, ?it/s]

In [10]:
b = pd.DataFrame(links_sunlight)

In [11]:
b = b.drop_duplicates()

In [12]:
b = b[0].tolist()

In [13]:
# Чтобы данные точно собрались нужно разделить ссылки на 100 чанков
chunks = np.array_split(b, 100)

In [16]:
n = 0
for i in range(0, 100):
    n+=1
    print(f'начало парсинга {n} части ссылок')
    df = sunlight_parser(chunks[i])
    df.to_csv(r'C:\Users\Trenkin.Sergey\Desktop\parsers\sunlight_current_data\\' + f'sunlight_{day_now}_{month_now}_{n}.csv', index=False)
    time.sleep(5)

начало парсинга 1 части ссылок


  0%|          | 0/146 [00:00<?, ?it/s]

начало парсинга 2 части ссылок


  0%|          | 0/146 [00:00<?, ?it/s]

начало парсинга 3 части ссылок


...

  0%|          | 0/145 [00:00<?, ?it/s]

начало парсинга 99 части ссылок


  0%|          | 0/145 [00:00<?, ?it/s]

начало парсинга 100 части ссылок


  0%|          | 0/145 [00:00<?, ?it/s]

In [17]:
os.chdir(r'C:\Users\Trenkin.Sergey\Desktop\parsers\sunlight_current_data')
extension = 'csv'
all_filenames = [i for i in glob.glob('*.{}'.format(extension))]
len(all_filenames)

100

In [18]:
data = pd.DataFrame()
for name  in all_filenames:
    df = pd.read_csv(name)
    data = pd.concat([df, data], ignore_index=True)

In [19]:
data

,date,competitor,name,group,params,price,article,url
0,14-05-2024,Sunlight,\n Золотая цепь\n,Цепи,"{'Модель': '342-00-0140-31130*', 'Материал изд...",836180,344746,https://sunlight.net/catalog/chains_344746.html
1,14-05-2024,Sunlight,\n Золотой браслет с фианитами\n,Браслеты,"{'Модель': 'бр081-20р-01*', 'Материал изделия'...",94710,344374,https://sunlight.net/catalog/bracelets_344374....
2,14-05-2024,Sunlight,\n Золотой браслет с фианитами\n,Браслеты,"{'Модель': 'бр081-20с-01*', 'Материал изделия'...",86410,344373,https://sunlight.net/catalog/bracelets_344373....
3,14-05-2024,Sunlight,\n Золотой браслет с бриллиантами\n,Браслеты,"{'Модель': '73314-B5W-01', 'Материал изделия':...",234990,986416,https://sunlight.net/catalog/bracelets_986416....
4,14-05-2024,Sunlight,\n Золотой браслет\n,Браслеты,"{'Модель': '47-00-0000-11176*', 'Материал изде...",17280,344031,https://sunlight.net/catalog/bracelets_344031....
...,...,...,...,...,...,...,...,...
14587,14-05-2024,Sunlight,\n Золотые серьги с бриллиантами\n,Серьги,"{'Модель': '0275-C5M-02', 'Материал изделия': ...",21990,55703,https://sunlight.net/catalog/earring_55703.html
14588,14-05-2024,Sunlight,\n Золотой крест-подвеска\n,Крест-подвеска,"{'Модель': '14592.1*', 'Материал изделия': 'Бе...",3990,288514,https://sunlight.net/catalog/pendants_288514.html
14589,14-05-2024,Sunlight,\n Золотое кольцо с фианитами\n,Кольца,"{'Модель': '20-02-0001-12622*', 'Материал изде...",17990,348664,https://sunlight.net/catalog/ring_261669.html
14590,14-05-2024,Sunlight,\n Золотое кольцо с сапфирами и бриллиантам...,Кольца,"{'Модель': '73061-K5R-01', 'Материал изделия':...",35990,341090,https://sunlight.net/catalog/ring_341090.html


In [20]:
data.to_csv(rf'C:\Users\Trenkin.Sergey\Desktop\parsers\sunlight_current_data\data_without_price.csv', index = False)

In [15]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14592 entries, 0 to 14591
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   date        14592 non-null  object
 1   competitor  14592 non-null  object
 2   name        14592 non-null  object
 3   group       14592 non-null  object
 4   params      14592 non-null  object
 5   price       14592 non-null  int64 
 6   article     14592 non-null  int64 
 7   url         14592 non-null  object
dtypes: int64(2), object(6)
memory usage: 912.1+ KB


In [16]:
data.duplicated().sum()

0

In [17]:
data = data.drop_duplicates()

In [18]:
data.duplicated().sum()

0

In [19]:
data['article'].isna().sum()

0

In [20]:
data = data.dropna()

In [21]:
data['article'] = data['article'].astype('int') 

In [25]:
test = np.array_split(data, 20)

In [26]:
n = 0
for i in range(n, 20):
    n += 1
    print(f'начало парсинга {n} части ссылок')
    df = get_personal_price(test[i].reset_index())
    df = pd.DataFrame(df)
    df.to_csv(r'C:\Users\Trenkin.Sergey\Desktop\parsers\sunlight_current_data\discounts\\' + f'sunlight_all_data_{day_now}_{month_now}_{n}.csv', index=False)
    time.sleep(5)

начало парсинга 1 части ссылок


100%|██████████| 730/730 [06:22<00:00,  1.91it/s]


начало парсинга 2 части ссылок


100%|██████████| 730/730 [06:25<00:00,  1.89it/s]


начало парсинга 3 части ссылок


...

100%|██████████| 729/729 [07:17<00:00,  1.67it/s]


начало парсинга 19 части ссылок


100%|██████████| 729/729 [07:10<00:00,  1.69it/s]


начало парсинга 20 части ссылок


100%|██████████| 729/729 [07:42<00:00,  1.58it/s]


In [27]:
os.chdir(r'C:\Users\Trenkin.Sergey\Desktop\parsers\sunlight_current_data\discounts')
extension = 'csv'
all_filenames = [i for i in glob.glob('*.{}'.format(extension))]
len(all_filenames)

20

In [28]:
data

,date,competitor,name,group,params,price,article,url
0,14-05-2024,Sunlight,\n Золотая цепь\n,Цепи,"{'Модель': '342-00-0140-31130*', 'Материал изд...",836180,344746,https://sunlight.net/catalog/chains_344746.html
1,14-05-2024,Sunlight,\n Золотой браслет с фианитами\n,Браслеты,"{'Модель': 'бр081-20р-01*', 'Материал изделия'...",94710,344374,https://sunlight.net/catalog/bracelets_344374....
2,14-05-2024,Sunlight,\n Золотой браслет с фианитами\n,Браслеты,"{'Модель': 'бр081-20с-01*', 'Материал изделия'...",86410,344373,https://sunlight.net/catalog/bracelets_344373....
3,14-05-2024,Sunlight,\n Золотой браслет с бриллиантами\n,Браслеты,"{'Модель': '73314-B5W-01', 'Материал изделия':...",234990,986416,https://sunlight.net/catalog/bracelets_986416....
4,14-05-2024,Sunlight,\n Золотой браслет\n,Браслеты,"{'Модель': '47-00-0000-11176*', 'Материал изде...",17280,344031,https://sunlight.net/catalog/bracelets_344031....
...,...,...,...,...,...,...,...,...
14587,14-05-2024,Sunlight,\n Золотые серьги с бриллиантами\n,Серьги,"{'Модель': '0275-C5M-02', 'Материал изделия': ...",21990,55703,https://sunlight.net/catalog/earring_55703.html
14588,14-05-2024,Sunlight,\n Золотой крест-подвеска\n,Крест-подвеска,"{'Модель': '14592.1*', 'Материал изделия': 'Бе...",3990,288514,https://sunlight.net/catalog/pendants_288514.html
14589,14-05-2024,Sunlight,\n Золотое кольцо с фианитами\n,Кольца,"{'Модель': '20-02-0001-12622*', 'Материал изде...",17990,348664,https://sunlight.net/catalog/ring_261669.html
14590,14-05-2024,Sunlight,\n Золотое кольцо с сапфирами и бриллиантам...,Кольца,"{'Модель': '73061-K5R-01', 'Материал изделия':...",35990,341090,https://sunlight.net/catalog/ring_341090.html


In [29]:
data_discount = pd.DataFrame()
for name  in all_filenames:
    df = pd.read_csv(name)
    data_discount = pd.concat([df, data_discount], ignore_index=True)

In [30]:
data = data.merge(data_discount, on='article', how='left')

In [31]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14592 entries, 0 to 14591
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   date            14592 non-null  object
 1   competitor      14592 non-null  object
 2   name            14592 non-null  object
 3   group           14592 non-null  object
 4   params          14592 non-null  object
 5   price_x         14592 non-null  int64 
 6   article         14592 non-null  int32 
 7   url             14592 non-null  object
 8   price_old       14282 non-null  object
 9   price_y         14590 non-null  object
 10  price_personal  1743 non-null   object
 11  discount        14592 non-null  object
dtypes: int32(1), int64(1), object(10)
memory usage: 1.3+ MB


In [32]:
data['discount'].value_counts()

discount
['Еще -50% на три, код: ТРИ до 17.05', 'Еще -40% на два, код: ДВА до 17.05', 'Еще -30% на одно, код: ОДИН до 17.05']                                 14057
['ХИТ-цена']                                                                                                                                           216
[nan]                                                                                                                                                  119
['Второе бесплатно, код:1=2 до 20.05']                                                                                                                 112
['ЕЩЕ -40% по коду: СЕГОДНЯ']                                                                                                                           81
['ХИТ-цена', 'Выгодно! Золото 5 290 р/г']                                                                                                                6
['Еще -50% на три, код: ТРИ до 17.05', 'Еще -40% на два, код:

In [33]:
data.head(20)

,date,competitor,name,group,params,price_x,article,url,price_old,price_y,price_personal,discount
0,14-05-2024,Sunlight,\n Золотая цепь\n,Цепи,"{'Модель': '342-00-0140-31130*', 'Материал изд...",836180,344746,https://sunlight.net/catalog/chains_344746.html,1 672 360 ₽,836 180 ₽,NaN,"['Еще -50% на три, код: ТРИ до 17.05', 'Еще -4..."
1,14-05-2024,Sunlight,\n Золотой браслет с фианитами\n,Браслеты,"{'Модель': 'бр081-20р-01*', 'Материал изделия'...",94710,344374,https://sunlight.net/catalog/bracelets_344374....,189 420 ₽,94 710 ₽,NaN,"['Еще -50% на три, код: ТРИ до 17.05', 'Еще -4..."
2,14-05-2024,Sunlight,\n Золотой браслет с фианитами\n,Браслеты,"{'Модель': 'бр081-20с-01*', 'Материал изделия'...",86410,344373,https://sunlight.net/catalog/bracelets_344373....,172 820 ₽,86 410 ₽,NaN,"['Еще -50% на три, код: ТРИ до 17.05', 'Еще -4..."
3,14-05-2024,Sunlight,\n Золотой браслет с бриллиантами\n,Браслеты,"{'Модель': '73314-B5W-01', 'Материал изделия':...",234990,986416,https://sunlight.net/catalog/bracelets_986416....,469 980 ₽,234 990 ₽,NaN,"['Еще -50% на три, код: ТРИ до 17.05', 'Еще -4..."
4,14-05-2024,Sunlight,\n Золотой браслет\n,Браслеты,"{'Модель': '47-00-0000-11176*', 'Материал изде...",17280,344031,https://sunlight.net/catalog/bracelets_344031....,34 560 ₽,17 280 ₽,NaN,"['Еще -50% на три, код: ТРИ до 17.05', 'Еще -4..."
5,14-05-2024,Sunlight,\n Золотое шейное украшение\n,Шейное украшение,"{'Модель': '6150456-45*', 'Страна производства...",62120,343992,https://sunlight.net/catalog/neck_decoration_3...,124 240 ₽,62 120 ₽,NaN,"['Еще -50% на три, код: ТРИ до 17.05', 'Еще -4..."
6,14-05-2024,Sunlight,\n Золотое шейное украшение с глиттером\n,Шейное украшение,"{'Модель': '12929-H5Y-01', 'Материал изделия':...",28990,986086,https://sunlight.net/catalog/neck_decoration_9...,57 980 ₽,28 990 ₽,NaN,"['Еще -50% на три, код: ТРИ до 17.05', 'Еще -4..."
7,14-05-2024,Sunlight,\n Золотой браслет\n,Браслеты,"{'Модель': '12932-B5Y-01', 'Материал изделия':...",57990,986078,https://sunlight.net/catalog/bracelets_986078....,115 980 ₽,57 990 ₽,NaN,"['Еще -50% на три, код: ТРИ до 17.05', 'Еще -4..."
8,14-05-2024,Sunlight,\n Золотое кольцо с бриллиантами\n,Кольца,"{'Модель': '71409-K5W-02', 'Материал изделия':...",99990,985936,https://sunlight.net/catalog/ring_985936.html,199 980 ₽,99 990 ₽,NaN,"['Еще -50% на три, код: ТРИ до 17.05', 'Еще -4..."
9,14-05-2024,Sunlight,\n Золотой браслет с бриллиантами\n,Браслеты,"{'Модель': '73353-B5R-01', 'Материал изделия':...",399990,985849,https://sunlight.net/catalog/bracelets_985849....,799 980 ₽,399 990 ₽,NaN,"['Второе бесплатно, код:1=2 до 20.05']"


In [34]:
data.to_csv(r'C:\Users\Trenkin.Sergey\Desktop\parsers\workspace\\' + f'sunlight_all_{day_now}_{month_now}_{datetime.today().year}.csv', index=False)

In [35]:
data.to_csv(rf'\\gold585.int\uk\Общее хранилище файлов\Служба аналитики\МЮР\parsed_data\sunlight_raw.csv', index=False)

In [36]:
def delete_files_in_folder(folder_path):
    for filename in os.listdir(folder_path):
        file_path = os.path.join(folder_path, filename)
        try:
            if os.path.isfile(file_path):
                os.remove(file_path)
        except Exception as e:
            print(f'Ошибка при удалении файла {file_path}. {e}')

In [38]:
delete_files_in_folder('C:\\Users\Trenkin.Sergey\Desktop\parsers\sunlight_current_data')
delete_files_in_folder('C:\\Users\Trenkin.Sergey\Desktop\parsers\sunlight_current_data\discounts')